In [ ]:
import cv2
import numpy as np
import skimage

def preprocess_page(page: np.ndarray, thresh=None):
    gray = cv2.cvtColor(page, cv2.COLOR_BGR2GRAY) if len(page.shape) == 3 else page

    if thresh is None:
        t, binary = cv2.threshold(
            gray,
            0,
            255,
            cv2.THRESH_BINARY + cv2.THRESH_OTSU
        )

    else:
        t = float(thresh)
        _, binary = cv2.threshold(
            gray,
            t,
            255,
            cv2.THRESH_BINARY
        )

    return binary, t

SyntaxError: invalid syntax (17942385.py, line 6)